In [1]:
%%sh
wget -N "https://sagemaker-sample-data-us-west-2.s3-us-west-2.amazonaws.com/autopilot/direct_marketing/bank-additional.zip"


--2023-12-17 17:38:44--  https://sagemaker-sample-data-us-west-2.s3-us-west-2.amazonaws.com/autopilot/direct_marketing/bank-additional.zip
Resolving sagemaker-sample-data-us-west-2.s3-us-west-2.amazonaws.com (sagemaker-sample-data-us-west-2.s3-us-west-2.amazonaws.com)... 52.92.132.82, 52.218.218.33, 52.92.194.250, ...
Connecting to sagemaker-sample-data-us-west-2.s3-us-west-2.amazonaws.com (sagemaker-sample-data-us-west-2.s3-us-west-2.amazonaws.com)|52.92.132.82|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 432828 (423K) [application/zip]
Saving to: ‘bank-additional.zip’

     0K .......... .......... .......... .......... .......... 11%  823K 0s
    50K .......... .......... .......... .......... .......... 23%  412K 1s
   100K .......... .......... .......... .......... .......... 35%  822K 0s
   150K .......... .......... .......... .......... .......... 47%  824K 0s
   200K .......... .......... .......... .......... .......... 59%  413K 0s
   250K .....

In [2]:
!unzip -o bank-additional.zip


Archive:  bank-additional.zip
   creating: bank-additional/
  inflating: bank-additional/bank-additional-names.txt  
  inflating: bank-additional/bank-additional.csv  
  inflating: bank-additional/bank-additional-full.csv  


In [4]:
import pandas as pd
data=pd.read_csv("bank-additional/bank-additional-full.csv")
data[:5]

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
1,57,services,married,high.school,unknown,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
2,37,services,married,high.school,no,yes,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
3,40,admin.,married,basic.6y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
4,56,services,married,high.school,no,no,yes,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no


In [5]:
import sagemaker
prefix = 'sagemaker/DEMO-smprocessing/input'
input_data = sagemaker.Session().upload_data(path='bank-additional/bank-additional-full.csv', key_prefix=prefix)

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


In [6]:
from sagemaker.sklearn.processing import SKLearnProcessor
role = sagemaker.get_execution_role()
sklearn_processor = SKLearnProcessor(framework_version='0.20.0',
 role=role,
 instance_type='ml.m5.xlarge',
 instance_count=1)


sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


In [7]:
!unzip bank-additional/SageMaker\ Lab.zip


unzip:  cannot find or open bank-additional/SageMaker Lab.zip, bank-additional/SageMaker Lab.zip.zip or bank-additional/SageMaker Lab.zip.ZIP.


In [10]:
from sagemaker.processing import ProcessingInput, ProcessingOutput
sklearn_processor.run(code='preprocessing.py',
 inputs=[ProcessingInput(
 source=input_data,
 destination='/opt/ml/processing/input')],
 outputs=[ProcessingOutput(output_name='train_data',
 source='/opt/ml/processing/train'),
 ProcessingOutput(output_name='test_data',
 source='/opt/ml/processing/test')]
 )

INFO:sagemaker:Creating processing-job with name sagemaker-scikit-learn-2023-12-17-17-40-49-864


........................./miniconda3/lib/python3.7/site-packages/sklearn/externals/joblib/externals/cloudpickle/cloudpickle.py:47: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  import imp
/miniconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:37: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  LARGE_SPARSE_SUPPORTED = LooseVersion(scipy_version) >= '0.14.0'
Reading input data from /opt/ml/processing/input/bank-additional-full.csv
Positive samples: 4639
Negative samples: 36537
Ratio: 7.88
Running preprocessing and feature engineering transformations
Train data shape after preprocessing: (32940, 58)
Test data shape after preprocessing: (8236, 58)
Saving training features to /opt/ml/processing/train/train_features.csv
Saving test features to /opt/ml/processing/test/test_features.csv
Saving training labels to /opt/ml/processing/train/train_labels.csv


In [11]:
preprocessing_job_description = sklearn_processor.jobs[-1].describe()
output_config =preprocessing_job_description['ProcessingOutputConfig']
for output in output_config['Outputs']:
    print(output['S3Output']['S3Uri'])

s3://sagemaker-us-east-1-187994925561/sagemaker-scikit-learn-2023-12-17-17-40-49-864/output/train_data
s3://sagemaker-us-east-1-187994925561/sagemaker-scikit-learn-2023-12-17-17-40-49-864/output/test_data


### Training a model using SageMaker SDK with built-in algorithm

####  Data preparation

In [19]:
import pandas as pd
dataset = pd.read_csv('HousingData.csv')
print(dataset)

        CRIM    ZN  INDUS  CHAS    NOX     RM   AGE     DIS  RAD  TAX  \
0    0.00632  18.0   2.31   0.0  0.538  6.575  65.2  4.0900    1  296   
1    0.02731   0.0   7.07   0.0  0.469  6.421  78.9  4.9671    2  242   
2    0.02729   0.0   7.07   0.0  0.469  7.185  61.1  4.9671    2  242   
3    0.03237   0.0   2.18   0.0  0.458  6.998  45.8  6.0622    3  222   
4    0.06905   0.0   2.18   0.0  0.458  7.147  54.2  6.0622    3  222   
..       ...   ...    ...   ...    ...    ...   ...     ...  ...  ...   
501  0.06263   0.0  11.93   0.0  0.573  6.593  69.1  2.4786    1  273   
502  0.04527   0.0  11.93   0.0  0.573  6.120  76.7  2.2875    1  273   
503  0.06076   0.0  11.93   0.0  0.573  6.976  91.0  2.1675    1  273   
504  0.10959   0.0  11.93   0.0  0.573  6.794  89.3  2.3889    1  273   
505  0.04741   0.0  11.93   0.0  0.573  6.030   NaN  2.5050    1  273   

     PTRATIO       B  LSTAT  MEDV  
0       15.3  396.90   4.98  24.0  
1       17.8  396.90   9.14  21.6  
2       17.8  3

In [39]:
dataset = pd.concat([dataset['MEDV'], dataset.drop(['MEDV'], axis=1)], axis=1)

In [42]:
# Assuming 'dataset' is your DataFrame
dataset['CRIM'].fillna(dataset['CRIM'].mean(), inplace=True)
dataset['ZN'].fillna(dataset['ZN'].mean(), inplace=True)
dataset['INDUS'].fillna(dataset['INDUS'].mean(), inplace=True)
dataset['CHAS'].fillna(dataset['CHAS'].mean(), inplace=True)
dataset['AGE'].fillna(dataset['AGE'].mean(), inplace=True)
dataset['LSTAT'].fillna(dataset['LSTAT'].mean(), inplace=True)


In [43]:
from sklearn.model_selection import train_test_split
training_dataset, validation_dataset = train_test_split(dataset, 
test_size=0.1)
print(training_dataset.shape)
print(validation_dataset.shape)

(455, 14)
(51, 14)


In [44]:
training_dataset.to_csv('training_dataset.csv', index=False, header=False)
validation_dataset.to_csv('validation_dataset.csv', index=False, header=False)

In [45]:
import sagemaker
print(sagemaker.__version__)
sess = sagemaker.Session()
bucket = sess.default_bucket()
prefix = 'boston-housing'
training_data_path = sess.upload_data(path='training_dataset.csv', 
key_prefix=prefix + '/input/training')
validation_data_path = sess.upload_data(path='validation_dataset.csv', 
key_prefix=prefix + '/input/validation')
print(training_data_path)
print(validation_data_path)


2.199.0
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml
s3://sagemaker-us-east-1-187994925561/boston-housing/input/training/training_dataset.csv
s3://sagemaker-us-east-1-187994925561/boston-housing/input/validation/validation_dataset.csv


#### 2.2 Configure the training job


In [46]:
import boto3
from sagemaker import image_uris
region = boto3.Session().region_name 
container = image_uris.retrieve('linear-learner', region)
print(container)


INFO:sagemaker.image_uris:Same images used for training and inference. Defaulting to image scope: inference.
INFO:sagemaker.image_uris:Defaulting to the only supported framework/algorithm version: 1.
INFO:sagemaker.image_uris:Ignoring unnecessary instance type: None.


382416733822.dkr.ecr.us-east-1.amazonaws.com/linear-learner:1


In [47]:
from sagemaker.estimator import Estimator
role = sagemaker.get_execution_role() 
ll_estimator = Estimator(container,
 role=role, 
 instance_count=1,
 instance_type='ml.m5.large',
 output_path='s3://{}/{}/output'.format(bucket, prefix)
)

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


In [48]:
ll_estimator.set_hyperparameters(
predictor_type='regressor',
mini_batch_size=32)


In [49]:
training_data_channel = sagemaker.TrainingInput(s3_data=training_data_path,content_type='text/csv')
validation_data_channel = sagemaker.TrainingInput(s3_data=validation_data_path,content_type='text/csv')
ll_data = {'train': training_data_channel, 'validation': validation_data_channel}


####  Running the training job

In [50]:
ll_estimator.fit(ll_data)

INFO:sagemaker:Creating training-job with name: linear-learner-2023-12-17-18-20-27-052


2023-12-17 18:20:27 Starting - Starting the training job...
2023-12-17 18:20:42 Starting - Preparing the instances for training......
2023-12-17 18:21:58 Downloading - Downloading input data......
2023-12-17 18:22:43 Downloading - Downloading the training image......
2023-12-17 18:23:54 Training - Training image download completed. Training in progress..Docker entrypoint called with argument(s): train
Running default environment configuration script
[12/17/2023 18:23:59 INFO 140267146045248] Reading default configuration from /opt/amazon/lib/python3.8/site-packages/algorithm/resources/default-input.json: {'mini_batch_size': '1000', 'epochs': '15', 'feature_dim': 'auto', 'use_bias': 'true', 'binary_classifier_model_selection_criteria': 'accuracy', 'f_beta': '1.0', 'target_recall': '0.8', 'target_precision': '0.8', 'num_models': 'auto', 'num_calibration_samples': '10000000', 'init_method': 'uniform', 'init_scale': '0.07', 'init_sigma': '0.01', 'init_bias': '0.0', 'optimizer': 'auto', 'lo

### Deploy our model

In [51]:
from time import strftime, gmtime
timestamp = strftime('%d-%H-%M-%S', gmtime())
endpoint_name = 'linear-learner-demo-'+timestamp
print(endpoint_name)


linear-learner-demo-17-18-25-10


In [52]:
ll_predictor = ll_estimator.deploy(endpoint_name=endpoint_name, 
 initial_instance_count=1, 
instance_type='ml.t2.medium')

INFO:sagemaker:Creating model with name: linear-learner-2023-12-17-18-25-22-171
INFO:sagemaker:Creating endpoint-config with name linear-learner-demo-17-18-25-10
INFO:sagemaker:Creating endpoint with name linear-learner-demo-17-18-25-10


----------------------!

In [54]:
test_sample = '0.00632,18.00,2.310,0,0.5380,6.5750,65.20,4.0900,1,296.0,15.30,4.98'

#ll_predictor.content_type = 'text/csv'
ll_predictor.serializer = sagemaker.serializers.CSVSerializer()
ll_predictor.deserializer = sagemaker.deserializers.CSVDeserializer()
response = ll_predictor.predict(test_sample)
print(response)

ModelError: An error occurred (ModelError) when calling the InvokeEndpoint operation: Received client error (400) from primary with message "unable to evaluate payload provided". See https://us-east-1.console.aws.amazon.com/cloudwatch/home?region=us-east-1#logEventViewer:group=/aws/sagemaker/Endpoints/linear-learner-demo-17-18-25-10 in account 187994925561 for more information.

In [55]:
response = ll_predictor.predict(test_sample)
print(response)

ModelError: An error occurred (ModelError) when calling the InvokeEndpoint operation: Received client error (400) from primary with message "unable to evaluate payload provided". See https://us-east-1.console.aws.amazon.com/cloudwatch/home?region=us-east-1#logEventViewer:group=/aws/sagemaker/Endpoints/linear-learner-demo-17-18-25-10 in account 187994925561 for more information.

In [ ]:
runtime = boto3.Session().client(service_name='runtime.sagemaker') 
response = runtime.invoke_endpoint(EndpointName=endpoint_name, 
 ContentType='text/csv', 
Body=test_sample)
print(response['Body'].read())